In [ ]:
import openai
import backoff
from bardapi import Bard
import csv
import random
import time


openai.api_key = ""
#not shown for security purposes 

@backoff.on_exception(backoff.expo, openai.error.RateLimitError, max_tries = 5)
def get_answer_from_openai(prompt):
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature = 1.0,
        stop = None,
        # max_tokens = 200
    )
    result = completion['choices'][0]['message']['content']
    return result

def get_answer_from_gpt4(file, prompt):
    
    completion = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature = 1.0,
        stop = None,
        # max_tokens = 200
    )
    result = completion['choices'][0]['message']['content']
    return result


def get_answer_from_bard(prompt):
    bard_token_list =  []
    random.shuffle(bard_token_list)

    for bard_token in bard_token_list:
        try:
            for i in range(5):
                result = Bard(token=bard_token, timeout=30).get_answer(prompt)["content"]
                if ('Response Error:' not in result):
                    return result
        except:   
            print('no') 
            continue

In [ ]:
#gpt4 use code edited to only input prompt, not file
def get_answer_from_gpt4_promptonly(prompt):
    
    completion = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature = 1.0,
        stop = None,
        # max_tokens = 200
    )
    result = completion['choices'][0]['message']['content']
    return result

In [ ]:
accommodation_type = 'airbnb'
hotel = ['employee can speak english', 'golf course', 'parking lot', 'kind emloyee', 'romantic', 'big hotel', 'jacuzzi', 'renovated', 'family trip', 'spacious room', 'spa', 'discount', 'bar', 'contactless check in', 'cozy', 'air conditioning', 'late checkout', 'comfortable', 'infinity pool', 'honeymoon', 'safe', 'picture window', 'good airflow', 'in front of the bus station', 'good restaurants', 'kids club', 'close beach', 'shopping mall', 'city', 'resort', 'shuttle bus available', 'early check in', '24h front desk', 'small hotel', 'fast response', 'name value', 'hygenic', 'within walkable distance', 'breakfast', 'housekeeping every day', 'meticulous', 'amenities', 'cheap', 'close to the airport', 'sea view']
airbnb = ['parking lot', 'friendly host', 'romantic', 'spacious', 'jacuzzi', 'renovated', 'family trip', 'couples', 'spacious rooms', 'spa', 'discount', 'bar', 'contactless check in', 'cozy', 'air conditioning', 'late check out', 'confortable', 'swimming pool', 'honeymoon', 'safe neighborhood', 'picture window', 'good airlow', 'close to the bus station', 'michelin restaurant', 'close to the beach', 'shopping', 'city', 'flexible check in times', 'hot tub', '24h support', 'small house', 'hygenic', 'within walking distance', 'kitchen', '썬베드', 'amenities', 'cheap', 'close to the airport', 'beach']
japanese_ryokan = ['employee can speak english', 'parking lot', 'kind employee', 'romantic', 'japanese', 'hinoki', 'traditional', 'renovated', 'family trip', 'spacious room', 'spa', 'discount', 'contactless check in', 'cozy', 'air conditioning', 'late checkout', 'confortable', 'kaiseki', 'spa', 'hot pool', 'honeymoon', 'safe', 'tatami room', 'good airflow', 'in front of the bus station', 'good restaurants', 'shopping mall', 'city', 'shuttle bus available', 'early check in', '24h fron desk', 'hygenic', 'within walking distance', 'breakfast', 'clean', 'meticulous', 'like a hotel', 'cheap', 'close to the airport']

when changing accomodation types:
1) change [accommodation_type] above to the desired accomodation_type (hotel, airbnb, japanese_ryokan, etc.)
2) change [keywords] below to the list corresponding to the accomodation_type
    (different keywords for different accomodation_types)
3) if needed, add wanted keywords / remove keywords to produce a more focused output

In [ ]:
from tqdm.auto import tqdm
keywords = airbnb #change
#change keywords based on desired accomodation

@backoff.on_exception(backoff.expo, (openai.error.RateLimitError,
                                     openai.error.APIError,
                                     openai.error.ServiceUnavailableError,),
                      max_tries=5, max_value = 60)
def augment(number):
    augment_output = []
    
    for n in tqdm(range(number)):
        random.shuffle(keywords)
        #keyword shuffle needed as code will return outputs with too many keywords at the front if not shuffles
        keywords_joined = ','.join(keywords)

        output = get_answer_from_gpt4_promptonly(keywords_joined + "\n Use two or more words in the list above to generate 1 question that a tourist using the " + accommodation_type + "database " + accommodation_type +"may have. The question must be generated with a question mark at the end.")
        #adding details to prompt will produce a more detailed output (ex: when searching for hotels --> instead of "tourist using the..?" use "tourist planning to travel to Paris using the..?")
        output_data = [output]
        augment_output.append(output_data)
    return augment_output

Why we do not use '...produce n number of questions" and only produce 1 question at a time:

Increase in output quality (detail and less errors) if producing one question at a time like prompt above

In [ ]:
#save to csv code (append mode)
def save(data, filename):
    with open(filename, 'a', newline = '', encoding='utf8') as csvfile:
        writer = csv.writer(csvfile)
        for row in data:
            writer.writerow(row)

for i in range(repetition):

save(augment(number of desired outputs), 'name of csv file you wish output to be saved to')

range(repetition number, integer) X augment(integer) = total amount of questions produced 
(ex: range(4) and augment(500) = 4 x 500 = 2000)

This is separated due to RateLimitError

In [ ]:
for i in range(4):
    try:
        save(augment(500), 'csv_file_name.csv')
        print("waiting for next")
        time.sleep(60) #for RateLimitError, Increase time(seconds) if needed
        print("resuming")
    except:
        continue